In [1]:
# %matplotlib notebook

In [1]:
# 1) Wipe out all Python variables
%reset -f
# 2) Force Python’s garbage collector to run
import gc
gc.collect()
# 3) If you’re using PyTorch + CUDA, free any lingering GPU memory
import torch
torch.cuda.empty_cache()

import importlib
from libs import trades, plots, params
importlib.reload(trades)
importlib.reload(plots)
importlib.reload(params)

import math
import pandas as pd
from pandas import Timestamp
import numpy as np

import glob
import os
import datetime as dt
from datetime import datetime
import optuna
import json

import matplotlib.pyplot as plt
from IPython.display import display, clear_output, update_display

pd.set_option('display.max_columns', None)


/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ticker         = params.ticker
save_path      = params.save_path

results_folder = "optuna results"              
n_trials = 100

df_path = os.path.join(save_path, f"{ticker}_base.csv")
df = pd.read_csv(df_path, index_col=0, parse_dates=["datetime"])
df


,open,high,low,close,volume,ask,bid
datetime,,,,,,,
2014-04-03 10:42:00,28.6500,28.6500,28.6500,28.6500,2000.0,28.658595,28.641405
2014-04-03 10:43:00,28.6500,28.6500,28.6500,28.6500,2000.0,28.658595,28.641405
2014-04-03 11:04:00,28.6500,28.6500,28.6500,28.6500,11220.0,28.658595,28.641405
2014-04-03 11:05:00,28.6500,28.6500,28.6500,28.6500,4620.0,28.658595,28.641405
2014-04-03 11:34:00,28.5005,28.5005,28.5005,28.5005,3460.0,28.509050,28.491950
...,...,...,...,...,...,...,...
2025-06-18 23:55:00,173.9000,173.9445,173.8670,173.8681,3136.0,173.920300,173.815900
2025-06-18 23:56:00,173.8200,173.9500,173.7900,173.9000,183.0,173.952200,173.847800
2025-06-18 23:57:00,173.9500,173.9500,173.8600,173.8601,240.0,173.912300,173.807900


In [4]:
def optimiz_function(df,
                    min_prof_thr, 
                    max_down_prop, 
                    gain_tightening_factor, 
                    merging_retracement_thr, 
                    merging_time_gap_thr, 
                    smooth_win_sig, 
                    pre_entry_decay, 
                    short_penalty,
                    buy_threshold, 
                    trailing_stop_thresh 
                    ):
    
    # First, adjust the input DataFrame's timestamps.
    df_prep = trades.prepare_interpolate_data(df=df, 
                                              regular_start_shifted=params.regular_start_shifted,
                                              regular_start=params.regular_start, 
                                              regular_end=params.regular_end)

    full_sim_results = trades.run_trading_pipeline(df_prep=df_prep, 
                                              col_signal='signal_smooth',
                                              col_action='signal_action',
                                              min_prof_thr=min_prof_thr, 
                                              max_down_prop=max_down_prop, 
                                              gain_tightening_factor=gain_tightening_factor, 
                                              smooth_win_sig=smooth_win_sig, 
                                              pre_entry_decay=pre_entry_decay,
                                              short_penalty=short_penalty,
                                              buy_threshold=buy_threshold, 
                                              trailing_stop_thresh=trailing_stop_thresh, 
                                              merging_retracement_thr=merging_retracement_thr, 
                                              merging_time_gap_thr=merging_time_gap_thr)
    
    sum_returns = np.sum([res[2]['Strategy Return ($)'] for res in full_sim_results.values()]) # just to check and confirm the values
    mean_returns = np.mean([res[2]['Strategy Return ($)'] for res in full_sim_results.values()]) # the metric that we use as a reference

    return mean_returns, sum_returns


In [5]:
# mean_returns, sum_returns = optimiz_function(df=df,
#                             min_prof_thr=params.min_prof_thr_man, 
#                             max_down_prop=params.max_down_prop_man, 
#                             gain_tightening_factor=params.gain_tightening_factor_man,  
#                             merging_retracement_thr=params.merging_retracement_thr_man, 
#                             merging_time_gap_thr=params.merging_time_gap_thr_man, 
#                             smooth_win_sig=params.smooth_win_sig_man, 
#                             pre_entry_decay=params.pre_entry_decay_man, 
#                             short_penalty=params.short_penalty_man, 
#                             buy_threshold=params.buy_threshold_man, 
#                             trailing_stop_thresh=params.trailing_stop_thresh_man, 
#                             )
# mean_returns, sum_returns

Step B1: identify_trades_daily …
Step B2: add_trade_signal_to_results …
Step B3: simulate_trading …


(0.46912286931818187, 1321.0500000000002)

In [ ]:

# === Objective Function ===
def objective(trial):
    # Suggest parameters to test.
    hyperpar = {
        "min_prof_thr": trial.suggest_float("min_prof_thr", 0.1, 0.4),
        "max_down_prop": trial.suggest_float("max_down_prop", 0.2, 0.7),
        "gain_tightening_factor": trial.suggest_float("gain_tightening_factor", 0.1, 0.3),
        "merging_retracement_thr": trial.suggest_float("merging_retracement_thr", 0.7, 1),
        "merging_time_gap_thr": trial.suggest_float("merging_time_gap_thr", 0.6, 0.9),
        "smooth_win_sig": trial.suggest_int("smooth_win_sig", 5, 60),
        "pre_entry_decay": trial.suggest_float("pre_entry_decay", 0.001, 0.01),
        "short_penalty": trial.suggest_float("short_penalty", 0.001, 0.01),
        "buy_threshold": trial.suggest_float("short_penalty", 0.05, 0.5),
        "trailing_stop_thresh": trial.suggest_float("trailing_stop_thresh", 0.1, 0.5),
    }

    # Run your strategy simulation with the current set of parameters.
    mean_returns, sum_returns = optimiz_function(
        df=df,
        min_prof_thr=hyperpar["min_prof_thr"],
        max_down_prop=hyperpar["max_down_prop"],
        gain_tightening_factor=hyperpar["gain_tightening_factor"],
        merging_retracement_thr=hyperpar["merging_retracement_thr"],
        merging_time_gap_thr=hyperpar["merging_time_gap_thr"],
        smooth_win_sig=hyperpar["smooth_win_sig"],
        pre_entry_decay=hyperpar["pre_entry_decay"],
        short_penalty=hyperpar["short_penalty"]
        buy_threshold=hyperpar["buy_threshold"],
        trailing_stop_thresh=hyperpar["trailing_stop_thresh"]
    )
    
    return mean_returns



In [ ]:
# ----------------------------------------------------------
# create ONE figure
# ----------------------------------------------------------
fig, ax = plt.subplots(figsize=(7, 3))
line,   = ax.plot([], [], "bo-")
ax.set(xlabel="Trial #", ylabel="Objective",
       title="Optuna optimisation progress", xlim=(0, 1), ylim=(0, 1))
ax.grid(True)

handle = display(fig, display_id=True)   # show once and keep handle
plt.close(fig)                           # <── prevents the duplicate static copy

# ----------------------------------------------------------
# callback
# ----------------------------------------------------------
def live_plot_callback(study, trial):
    xs = [t.number   for t in study.trials]
    ys = [t.value    for t in study.trials]

    line.set_data(xs, ys)
    ax.set_xlim(-1, len(xs))
    ax.set_ylim(min(ys) * 0.95, max(ys) * 1.15)

    update_display(fig, display_id=handle.display_id)   # refresh only this figure




In [ ]:
# === Create and Run the Study ===
study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=n_trials, callbacks=[live_plot_callback])

# === Print Final Results ===
print("Best Parameters:", study.best_params)
print("Best Average Improvement:", study.best_value)


# ------------------------------------------------------------------
# Build a dynamic file-name:  <results_folder>/<TICKER>_<YYYYMMDD>-<YYYYMMDD>_optuna.json
# ------------------------------------------------------------------

start_date  = df.index.min().strftime("%Y%m%d")  # ❹ first date in the DataFrame
end_date    = df.index.max().strftime("%Y%m%d")  # ❺ last  date in the DataFrame

file_name = f"{ticker}_{start_date}-{end_date}_optuna_results.json"
file_path = os.path.join(results_folder, file_name)

# ------------------------------------------------------------------
# Dump study results
# ------------------------------------------------------------------
with open(file_path, "w") as f:
    json.dump(
        {
            "best_params": study.best_params,
            "best_value" : study.best_value,
            "trials": [
                {
                    "number" : t.number,
                    "value"  : t.value,
                    "params" : t.params,
                    "state"  : t.state.name
                }
                for t in study.trials
            ],
        },
        f,
        indent=4,
    )

print(f"Optuna results saved to: {file_path}")
